In [1]:
import pandas as pd

# Tải dữ liệu từ file CSV
file_path = 'combined_data_safe.csv'
data = pd.read_csv(file_path)

# Chuyển đổi timestamps từ Unix sang datetime
data['datetime'] = pd.to_datetime(data['timestamp'], unit='s')

# Tạo đặc trưng thời gian: giờ trong ngày
data['hour'] = data['datetime'].dt.hour

# Giả sử giờ hoạt động của tòa nhà là từ 8:00 sáng đến 6:00 chiều (8 - 18 giờ)
start_hour = 8
end_hour = 18

# Lọc dữ liệu chỉ trong khoảng giờ hoạt động
operating_hours_data = data[(data['hour'] >= start_hour) & (data['hour'] <= end_hour)]

# In ra một số dòng dữ liệu để kiểm tra
operating_hours_data.head()


,timestamp,oa_temp,room_temp_Mf-1-1,room_temp_Mf-1-2,room_temp_Mf-1-3,room_temp_Mf-2-1-1,room_temp_Mf-2-1-2,room_temp_Mf-2-1-3,room_temp_Mf-2-1-4,room_temp_Ac-2-1,...,dew_temp_Ac-2-3,dew_temp_Ac-2-4,dew_temp_Ac-2-5,fan_stat_Ac-2-1,fan_stat_Ac-2-2,fan_stat_Ac-2-3,fan_stat_Ac-2-4,fan_stat_Ac-2-5,datetime,hour
24,1699603748,18.5,20.3,20.2,20.8,22.6,22.8,21.2,20.9,21.1,...,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,2023-11-10 08:09:08,8
25,1699604948,19.3,20.4,20.4,20.9,22.6,22.8,21.3,21.0,21.2,...,15.4,15.4,15.4,0.0,0.0,0.0,0.0,0.0,2023-11-10 08:29:08,8
26,1699606148,20.4,20.5,20.5,21.0,22.6,22.8,21.5,21.2,21.4,...,15.8,15.8,15.8,0.0,0.0,0.0,0.0,0.0,2023-11-10 08:49:08,8
27,1699607348,22.0,20.6,20.7,21.1,22.9,22.9,21.8,21.3,21.6,...,16.5,16.5,16.5,0.0,0.0,0.0,0.0,0.0,2023-11-10 09:09:08,9
28,1699608548,23.3,20.8,20.9,21.3,23.1,23.1,22.0,21.5,21.9,...,16.8,16.8,16.8,0.0,0.0,0.0,0.0,0.0,2023-11-10 09:29:08,9


In [2]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Chọn một cột dữ liệu nhiệt độ để phân tích, ví dụ: 'room_temp_Mf-1-1'
temperature_data = operating_hours_data['room_temp_Mf-1-1']

# Xác định ngưỡng nhiệt độ lý tưởng
ideal_temp_low = 20 - 2
ideal_temp_high = 20 + 2

# Huấn luyện mô hình ARIMA
# Lưu ý: Các tham số (p, d, q) của ARIMA cần được xác định dựa trên dữ liệu cụ thể
# Ví dụ: sử dụng p=5, d=1, q=0
model = ARIMA(temperature_data, order=(5, 1, 0))
model_fit = model.fit()

# Dự đoán giá trị nhiệt độ
predictions = model_fit.forecast(steps=len(temperature_data))

# Phát hiện bất thường: những điểm dữ liệu nào nằm ngoài ngưỡng lý tưởng
anomalies = np.where((temperature_data < ideal_temp_low) | (temperature_data > ideal_temp_high), True, False)

# Tìm những thời điểm có bất thường
anomalous_timestamps = operating_hours_data['datetime'][anomalies]

# In ra các thời điểm bất thường
print("Anomalous timestamps:")
print(anomalous_timestamps)



c:\Users\phamh\.conda\envs\chart2text\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\phamh\.conda\envs\chart2text\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\phamh\.conda\envs\chart2text\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\Users\phamh\.conda\envs\chart2text\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsu

Anomalous timestamps:
36    2023-11-10 12:09:08
37    2023-11-10 12:29:08
38    2023-11-10 12:49:08
39    2023-11-10 13:09:08
40    2023-11-10 13:29:08
41    2023-11-10 13:49:08
42    2023-11-10 14:09:08
43    2023-11-10 14:29:08
44    2023-11-10 14:49:08
45    2023-11-10 15:09:08
46    2023-11-10 15:29:08
47    2023-11-10 15:49:08
48    2023-11-10 16:09:08
49    2023-11-10 16:29:08
50    2023-11-10 16:49:08
51    2023-11-10 17:09:08
52    2023-11-10 17:29:08
53    2023-11-10 17:49:08
54    2023-11-10 18:09:08
55    2023-11-10 18:29:08
56    2023-11-10 18:49:08
96    2023-11-11 08:09:08
97    2023-11-11 08:29:08
98    2023-11-11 08:49:08
99    2023-11-11 09:09:08
100   2023-11-11 09:29:10
101   2023-11-11 09:49:08
102   2023-11-11 10:09:08
103   2023-11-11 10:29:08
104   2023-11-11 10:49:08
105   2023-11-11 11:09:08
106   2023-11-11 11:29:08
107   2023-11-11 11:49:08
108   2023-11-11 12:09:08
109   2023-11-11 12:29:08
110   2023-11-11 12:49:08
111   2023-11-11 13:09:08
112   2023-11-11

c:\Users\phamh\.conda\envs\chart2text\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split

# Chọn và chuẩn bị dữ liệu
features = ['room_temp_Ac-2-1', 'slab_temp_Ac-2-1', 'oa_temp', 'fan_stat_Ac-2-1', 'dew_temp_Ac-2-1']
data_for_model = operating_hours_data[features].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_model)

# Tạo chuỗi dữ liệu
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length, 0]  # Giả sử chúng ta muốn dự đoán nhiệt độ phòng
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 5
X, y = create_sequences(scaled_data, seq_length)

# Chia dữ liệu thành tập huấn luyện và kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, len(features))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Dự đoán và phân tích bất thường có thể được thực hiện sau khi huấn luyện mô hình


Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: nan
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: nan
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 9/100
4/4 [==============================] - 0s 6ms/step - loss: nan
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 11/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 12/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 13/100
4/4 [==============================] - 0s 6ms/step - loss: nan
Epoch 14/100
4/4 [===

In [12]:
import matplotlib.pyplot as plt
# Bước 1: Dự đoán
predictions = model.predict(X_test)

# Bước 2: Tính toán sai số
mse = np.mean(np.power(y_test - predictions, 2), axis=1)

# Bước 3: Xác định ngưỡng bất thường
# Chúng ta có thể xác định ngưỡng này dựa trên phân phối của sai số hoặc dựa trên kinh nghiệm
mse_threshold = np.percentile(mse, 95)  # ví dụ: lấy ngưỡng ở phần trăm thứ 95 của sai số

# Phát hiện bất thường
anomalies = mse > mse_threshold

# Bước 4: Trực quan hóa
plt.figure(figsize=(15, 6))
plt.plot(y_test, label='Actual Temperature')
plt.plot(predictions, label='Predicted Temperature')
plt.scatter(np.where(anomalies), y_test[anomalies], color='r', label='Anomaly')
plt.title('Temperature Prediction and Anomalies')
plt.xlabel('Time Step')
plt.ylabel('Temperature')
plt.legend()
plt.show()


ImportError: Matplotlib requires numpy>=1.21; you have 1.20.3